In [2]:
import ee
ee.Authenticate()
# Initialize Earth Engine



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [12]:


# Define the Bounding Box (BBOX) as [min_lon, min_lat, max_lon, max_lat]
#bbox = ee.Geometry.BBox(-113.5, 38.8, -113.0, 39.2)  # Example for Pahvant Valley

# xmin, ymin, xmax, ymax
box = [-110.75, 39.43, -110.7, 39.46]

bbox = ee.Geometry.BBox(*box)
# Load the Landsat 8 image collection
# Load Landsat 8 TOA collection
landsat = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
    .filterBounds(bbox)
    .filterDate("2023-06-01", "2023-06-30")
    .sort("CLOUD_COVER")
    .first()
)

# Select only the 30m bands to avoid projection issues
bands_30m = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # Excludes B8 (15m) & B10/B11 (100m)
landsat = landsat.select(bands_30m)

# Convert all bands to Float32 to ensure consistency
landsat = landsat.toFloat()

# Get the original projection and scale
orig_proj = landsat.select('B1').projection()
orig_scale = orig_proj.nominalScale()

# Reproject to preserve the Landsat grid
reprojected_image = landsat.reproject(crs=orig_proj, scale=orig_scale)



# Define export parameters
export_task = ee.batch.Export.image.toDrive(
    image=reprojected_image,
    description="Landsat_Grid_Preserved",
    folder="EarthEngineExports",
    fileNamePrefix="landsat_grid_sample",
    scale=30,  # Landsat native resolution
    region=bbox.bounds().getInfo()["coordinates"],  # Use bounding box
    fileFormat="GeoTIFF",
    crs=orig_proj.getInfo()["crs"]  # Use original Landsat CRS
)

# Start the export task
export_task.start()

print("Export started. Check Google Drive for the image.")


Export started. Check Google Drive for the image.


In [14]:
print(export_task.status())


{'state': 'RUNNING', 'description': 'Landsat_Grid_Preserved', 'priority': 100, 'creation_timestamp_ms': 1738592748000, 'update_timestamp_ms': 1738592750883, 'start_timestamp_ms': 1738592750852, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'L7JJUUL6F7M2WGKMDVX2JOW5', 'name': 'projects/114443260784/operations/L7JJUUL6F7M2WGKMDVX2JOW5'}


In [10]:
landsat